# Maritime Graph Weighting and Pathfinding Pipeline

This notebook provides a comprehensive, end-to-end workflow for processing a maritime navigation graph. It covers:
1.  **Conversion**: Converting an undirected graph to a directed one.
2.  **Enrichment**: Adding S-57 feature data (depth, clearance, orientation) to graph edges.
3.  **Weighting**: Applying a three-tier system of static, directional, and dynamic (vessel-specific) weights.
4.  **Pathfinding**: Calculating an optimal route on the fully weighted graph.

The entire pipeline is optimized for large graphs by performing all intensive operations directly within the PostGIS database. The graph is only loaded into memory at the final pathfinding step.

## 1. Notebook Settings

Adjust the parameters in this section to control the notebook's behavior. You can run the entire pipeline or toggle individual steps.

In [ ]:
# --- Graph Configuration ---
graph_name_undirected = "h3_graph_pg_6_11" # Source (undirected) graph
graph_name_directed = "h3_graph_directed_pg_6_11_v3" # Target for directed, weighted graph

# --- Workflow Control ---
# Set these to True or False to enable/disable steps.
# For a full run, set all to True. For a partial run, disable completed steps.
workflow_steps = {
    "run_conversion_to_directed": True, # Creates the new directed graph tables
    "run_enrichment": True,             # Adds S-57 feature data (ft_*) to edges. REQUIRED for all weighting.
    "run_static_weights": True,         # Applies weights from static layers (land, fairways, etc.)
    "run_directional_weights": True,    # Applies weights based on traffic flow (TSS, fairways)
    "run_dynamic_weights": True,        # Applies final vessel-specific weights (draft, height)
    "run_pathfinding": True             # Loads the final graph and calculates a route
}

# --- Vessel & Environment Parameters (for Dynamic Weights & Pathfinding) ---
vessel_params = {
 'draft': 7.5,           # meters
 'height': 30.0,         # meters (for overhead clearance)
 'safety_margin': 2.0,   # meters (for under-keel clearance)
 'vessel_type': 'cargo'
}

env_conditions = {
 'weather_factor': 1.2,      # 1.0=good, >1.0=poor
 'visibility_factor': 1.1,   # 1.0=good, >1.0=poor
 'time_of_day': 'day'        # 'day' or 'night'
}

# --- Pathfinding Ports ---
departure_port_name = "SF Pilot"
arrival_port_name = "San Francisco Arrival"

departure_coords = {"lon": -122.780, "lat": 37.006}
arrival_coords = {"lon": -122.400, "lat": 37.805}

## 2. Setup and Initialization

This section imports all necessary libraries and initializes the core classes (`ENCDataFactory`, `H3Graph`, `Weights`).

In [ ]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
import geopandas as gpd
from shapely.geometry import Point

# Add the project root to the Python path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.maritime_module.core.graph import H3Graph, Weights, FineTuning
from src.maritime_module.core.s57_data import ENCDataFactory
from src.maritime_module.core.pathfinding_lite import Route
from src.maritime_module.utils.port_utils import PortData

# Load environment variables from .env file
load_dotenv(project_root / ".env")

output_dir = Path.cwd() / 'output'
output_dir.mkdir(exist_ok=True)

# --- Database Connection ---
db_params = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

# --- Initialize Core Classes ---
factory = ENCDataFactory(source=db_params, schema="us_enc_all")
h3 = H3Graph(data_factory=factory, route_schema_name="routes", graph_schema_name="graph")
weights_manager = Weights(data_factory=factory)
port_manager = PortData()

print("Setup complete. Core classes initialized.")

## 3. Convert to Directed Graph

This step converts the undirected source graph into a directed graph by creating bidirectional edges. This is a prerequisite for applying directional weights. The operation is performed entirely on the database side for maximum performance.

In [ ]:
if workflow_steps["run_conversion_to_directed"]:
    print(f"Converting '{graph_name_undirected}' to directed graph '{graph_name_directed}'...")
    h3.convert_to_directed_postgis(
        source_table_prefix=graph_name_undirected,
        target_table_prefix=graph_name_directed,
        edges_schema="graph",
        drop_existing=True  # Ensures a clean start for the pipeline
    )
    print("\nConversion to directed graph complete.")
else:
    print("Skipping conversion to directed graph.")

## 4. Enrich Edges with S-57 Features

This is a **critical prerequisite** for all weighting steps. It performs a spatial join in PostGIS to extract key attributes (depth, clearance, orientation, etc.) from S-57 layers and adds them as `ft_*` columns to the graph's edge table. This step can be time-consuming but only needs to be run once.

In [ ]:
if workflow_steps["run_enrichment"]:
    # First, we need the boundary of the graph to find relevant ENCs
    print("Loading graph boundary to determine relevant ENCs...")
    try:
        # We only need the nodes to calculate the boundary, not the full graph
        nodes_df = gpd.read_postgis(f'SELECT geometry FROM graph."{graph_name_directed}_nodes"', factory.manager.engine, geom_col='geometry')
        graph_boundary = nodes_df.unary_union.convex_hull
        enc_list = factory.get_encs_by_boundary(graph_boundary)
        print(f"Found {len(enc_list)} ENCs intersecting with the graph boundary.")

        # Get the feature layers to extract from the S57 classifier
        feature_layers_to_enrich = weights_manager.get_feature_layers_from_classifier()

        # Run the server-side enrichment process
        summary = weights_manager.enrich_edges_with_features_postgis(
            enc_names=enc_list,
            schema_name="graph",
            graph_name=graph_name_directed,
            enc_schema="us_enc_all",
            feature_layers=feature_layers_to_enrich,
            is_directed=True, # IMPORTANT: Ensures features are propagated to reverse edges
            include_sources=False,
            soundg_buffer_meters=30
        )
        print("\nEnrichment complete. Summary:")
        print(summary)
    except Exception as e:
        print(f"An error occurred during enrichment: {e}")
        print("Please ensure the directed graph tables exist.")
else:
    print("Skipping edge enrichment.")

## 5. Apply Static Weights

This step applies weights based on proximity to static features like land, fairways, and obstructions. It uses a distance-based degradation model defined in the `graph_config.yml` file.

In [ ]:
if workflow_steps["run_static_weights"]:
    print("Applying static weights...")
    # The list of ENCs and static layers are read from the config file internally
    nodes_df = gpd.read_postgis(f'SELECT geometry FROM graph."{graph_name_directed}_nodes"', factory.manager.engine, geom_col='geometry')
    graph_boundary = nodes_df.unary_union.convex_hull
    enc_list = factory.get_encs_by_boundary(graph_boundary)
    config = weights_manager._load_config()

    weights_manager.apply_static_weights_postgis(
        graph_name=graph_name_directed,
        enc_names=enc_list,
        schema_name="graph",
        enc_schema="us_enc_all",
        static_layers=config["weight_settings"]["static_layers"],
        usage_bands=[3, 4, 5] # Focus on higher-detail bands for static features
    )
    print("Static weights applied successfully.")
else:
    print("Skipping static weight application.")

## 6. Apply Directional Weights

This step calculates penalties and rewards based on the alignment of an edge with the intended traffic flow of features like Traffic Separation Schemes (TSS) and fairways. It uses the `ft_orient` and `ft_trafic` attributes extracted during enrichment.

In [ ]:
if workflow_steps["run_directional_weights"]:
    print("Applying directional weights...")
    # The configuration (angle bands, layers) is read from graph_config.yml internally
    config = weights_manager._load_config()
    directional_config = config["weight_settings"]["directional_weights"]
    two_way_config = directional_config.get("two_way_traffic", {})

    weights_manager.calculate_directional_weights_postgis(
        schema_name="graph",
        graph_name=graph_name_directed,
        apply_to_layers=directional_config.get("apply_to_layers"),
        angle_bands=directional_config.get("angle_bands"),
        two_way_enabled=two_way_config.get("enabled", True),
        reverse_check_threshold=two_way_config.get("reverse_check_threshold", 95)
    )
    print("Directional weights applied successfully.")
else:
    print("Skipping directional weight application.")

## 7. Apply Dynamic (Vessel-Specific) Weights

This is the final weighting step. It combines all previously calculated weights (static, directional) with vessel-specific constraints (draft, height) to produce the final `adjusted_weight` for each edge. This `adjusted_weight` is what the pathfinding algorithm will use.

In [ ]:
if workflow_steps["run_dynamic_weights"]:
    print("Calculating final dynamic weights...")
    weights_manager.calculate_dynamic_weights_postgis(
        graph_name=graph_name_directed,
        schema_name="graph",
        vessel_parameters=vessel_params,
        environmental_conditions=env_conditions,
    )
    print("Dynamic weights calculated successfully.")
else:
    print("Skipping dynamic weight calculation.")

## 8. Pathfinding and Analysis

With the graph fully weighted, this final step loads the graph into memory and calculates an optimal route between a departure and arrival point. The route is then saved to a file for visualization in a GIS application.

In [ ]:
if workflow_steps["run_pathfinding"]:
    print("--- Starting Pathfinding ---")
    # Create or update the custom port locations
    port_manager.create_custom_port(
        port_name=departure_port_name,
        lon=departure_coords['lon'],
        lat=departure_coords['lat'],
        if_exists='update'
    )
    port_manager.create_custom_port(
        port_name=arrival_port_name,
        lon=arrival_coords['lon'],
        lat=arrival_coords['lat'],
        if_exists='update'
    )

    # Get port geometries
    departure_port = port_manager.get_port_by_name(departure_port_name)
    arrival_port = port_manager.get_port_by_name(arrival_port_name)

    if not departure_port or not arrival_port:
        print("Error: Could not find departure or arrival port.")
    else:
        # Load the fully weighted graph from PostGIS into memory
        print(f"Loading final weighted graph '{graph_name_directed}' from PostGIS...")
        G = h3.load_graph_from_postgis(graph_name_directed)
        print(f"Graph loaded with {G.number_of_nodes():,} nodes and {G.number_of_edges():,} edges.")

        # Initialize the routing engine
        route_finder = Route(graph=G, data_manager=factory.manager)

        # Calculate the detailed route using the 'adjusted_weight'
        print(f"Calculating route from {departure_port_name} to {arrival_port_name}...")
        route_detail = route_finder.detailed_route(
            departure_point=departure_port.geometry,
            arrival_point=arrival_port.geometry,
            weight_column='adjusted_weight' # CRITICAL: Use the final calculated weight
        )

        # Save the route to a file for visualization
        output_path = output_dir / f"detailed_route_{vessel_params['draft']}m_draft.geojson"
        route_finder.save_detailed_route_to_file(
            route_detail,
            output_path=str(output_path)
        )
        print(f"\nPathfinding complete. Route saved to: {output_path}")
else:
    print("Skipping pathfinding.")

## 9. (Optional) Export Weighted Graph

If you need to use the final weighted graph in another application (like QGIS), you can export it from PostGIS to a GeoPackage file. This is much more efficient than loading it into memory first.

In [ ]:
# Set this to True if you want to export the final graph
run_export = False

if run_export:
    output_gpkg_path = output_dir / f"{graph_name_directed}.gpkg"
    print(f"Exporting '{graph_name_directed}' from PostGIS to '{output_gpkg_path}'...")

    # Use the efficient ogr2ogr-based export method
    try:
        h3.export_postgis_to_gpkg(
            schema_name="graph",
            graph_name=graph_name_directed,
            output_path=str(output_gpkg_path)
        )
        print("Export complete.")
    except FileExistsError:
        print(f"Export failed: File already exists at {output_gpkg_path}. Please delete it and try again.")
    except Exception as e:
        print(f"An error occurred during export: {e}")
else:
    print("Skipping graph export.")